"do you have a sequence comparison of all the haplotypes for the best AAT candidate gene that we identified in the blueberry paper? Like in Genious or some other visualization tool that would show the potential nucleotide variants responsible for the trait?"


"Yes, there was one contig only that appears to have the variant. I don’t actually know which SNP, but based on Andrew, it was VcAAT1a on contig h2tg000453l_1. I have the gene sequence, but also can do some comparison looking for the SNP. "


## Goal
create alignment of all seqs that are similar to h2tg000453l_1 contig from M7H2 assembly. 

## Steps
1) Map all contigs to h2tg000453l_1.
2) Filter alignment to include only contigs that map well. Rename seqs in alignments to tell which haplotype the contig came from. 
3) Look for SNPs in the alignment. 

In [1]:
WKDIR=/workspace/hraijc/Blueberry/Blueberry_Antho_QLT/17022025_SNP_Search
CONTIG_DIR=/output/genomic/plant/Vaccinium/corymbosum/2023-10-18_M7xNui_TrioBinned_HiFi_Assemblies/Classified_Reads_Only_Assemblies
M7H1_CONTIGS=${CONTIG_DIR}/M7_Haplotypes/Purged_Assemblies/classified_M7_hap1.purged.fa	
M7H2_CONTIGS=${CONTIG_DIR}/M7_Haplotypes/Purged_Assemblies/classified_M7_hap2.purged.fa	
NuiH1_CONTIGS=${CONTIG_DIR}/Nui_Haplotypes/Purged_Assemblies/classified_Nui_hap1.purged.fa
NuiH2_CONTIGS=${CONTIG_DIR}/Nui_Haplotypes/Purged_Assemblies/classified_Nui_hap2.purged.fa

h2tg453=${WKDIR}/h2tg000453l_1.fa


mkdir -p $WKDIR/log
cd ${WKDIR}

In [2]:
#Names of contigs will be tricky. Will need to sort this out later.
head $NuiH1_CONTIGS -n 1
head $M7H1_CONTIGS -n 1

>h1tg000001l_1
>h1tg000001l_1


In [3]:
sed -e 's/>/>M7_/g' $M7H1_CONTIGS >> $WKDIR/Renamed_CONTIGS.fa
sed -e 's/>/>M7_/g' $M7H2_CONTIGS >> $WKDIR/Renamed_CONTIGS.fa
sed -e 's/>/>Nui_/g' $NuiH1_CONTIGS >> $WKDIR/Renamed_CONTIGS.fa
sed -e 's/>/>Nui_/g' $NuiH2_CONTIGS >> $WKDIR/Renamed_CONTIGS.fa


In [4]:
grep ">" $WKDIR/Renamed_CONTIGS.fa | tail

>Nui_h2tg000684l_1
>Nui_h2tg000696l_1
>Nui_h2tg000698l_1
>Nui_h2tg000699l_1
>Nui_h2tg000701l_1
>Nui_h2tg000705l_1
>Nui_h2tg000706l_1
>Nui_h2tg000709l_1
>Nui_h2tg000710l_1
>Nui_h2tg000711l_1


In [8]:
#Pull out contig with causitive variant
module load seqkit

seqkit grep -p "h2tg000453l_1" ${M7H2_CONTIGS} -o ${h2tg453}

In [18]:
#Its quite short
seqkit stats  ${h2tg453}

file                                                                                  format  type  num_seqs  sum_len  min_len  avg_len  max_len
/workspace/hraijc/Blueberry/Blueberry_Antho_QLT/17022025_SNP_Search/h2tg000453l_1.fa  FASTA   DNA          1  161,688  161,688  161,688  161,688


In [5]:
# Map contig to m7h1 contigs 
module load minimap2/2.22
sbatch << EOF
#!/bin/bash
#SBATCH -J mapping
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=5G
#SBATCH --time=01:10:00
#SBATCH -o $WKDIR/log/%J.out
#SBATCH -e $WKDIR/log/%J.err


#Map to Reference Assembly
minimap2 -x asm5 ${h2tg453} $WKDIR/Renamed_CONTIGS.fa --secondary=no -t 4 -o Var2Contigs.paf

EOF


Submitted batch job 8377540


In [6]:
#Filter hits 
cat Var2Contigs.paf | awk '$10 >= 1000' > Var2Contigs_min1k.paf


In [9]:
#Pull out hits
module load seqkit
cat Var2Contigs_min1k.paf | cut -f 1 | sort | uniq > Var2Contigs_min1k.txt
seqkit grep -f Var2Contigs_min1k.txt $WKDIR/Renamed_CONTIGS.fa -o Var2Contigs_min1k.fa

In [10]:
#Map again in sam format output
# Map contig to m7h1 contigs 
module load minimap2/2.22
sbatch << EOF
#!/bin/bash
#SBATCH -J mapping
#SBATCH --cpus-per-task=4
#SBATCH --mem-per-cpu=5G
#SBATCH --time=01:10:00
#SBATCH -o $WKDIR/log/%J.out
#SBATCH -e $WKDIR/log/%J.err


#Map to Reference Assembly
minimap2 -x asm5 ${h2tg453} Var2Contigs_min1k.fa --secondary=no -t 4 -a -o Var2Contigs_min1k.sam

EOF


Submitted batch job 8377652


In [33]:
grep "h2tg000453l_1" W85_20_p0.mRNAlift.gmap.M7h2.gff3 > h2tg000453l_1.gff3